# Elasticity equation

In this tutorial we present how to solve the elasticity equation with [PyGeoN](https://github.com/compgeo-mox/pygeon) and [PorePy](https://github.com/pmgbergen/porepy).  The unknown is the displacement $u$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, we want to solve the following problem: find $u$ such that
$$
\nabla \cdot [ 2 \mu \epsilon(u) + \lambda \nabla \cdot u] = -b
$$
with $\epsilon$ the symmetric gradient and $b$ a body force.

We will use the Multi-Point Stress Approximation (MPSA) to discretise the problem.

## Exercise 1: rigid-body motion

In the rigid-body motion the map $\chi$ is a composition of a rotation and a traslation.
For this test case we set $\Omega = [0, 1]^2$, $b = 0$, and the following boundary conditions:
$$
u(x) = (R - I)x \text{ on } \partial \Omega 
$$
with $R$ a rotation matrix of angle $\theta$.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [2]:
mesh_size = 0.05
dim = 2

sd = pg.unit_grid(dim, mesh_size, as_mdg=False)
sd.compute_geometry()

With the following code we set the data, in particular the Lamé and the Kirchhoff modulus, and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "elasticity"

lambda_ = 1
mu = 0.5
theta = 1e-3  # 1e-1 1e-3

# it's a vector field the displacement
b_nodes = np.hstack([sd.tags["domain_boundary_nodes"]] * dim)

R = np.array(
    [
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)],
    ]
)
bc_fun = lambda x: (R - np.eye(dim)) @ x[:2]

Let us now put all the data together.

In [4]:
lambda_ = lambda_ * np.ones(sd.num_cells)
mu = mu * np.ones(sd.num_cells) / 2
C = pp.FourthOrderTensor(mu, lambda_)

# Define boundary type
b_faces = sd.get_all_boundary_faces()
num_b_faces = b_faces.size
labels = np.array(["dir"] * num_b_faces)

bound = pp.BoundaryConditionVectorial(sd, b_faces, labels)

bc_values = bc_fun(sd.face_centers)
bc_values = bc_values.ravel("F")

# No source term
source = np.zeros(sd.num_cells * sd.dim)

# collect all data
data = {
    pp.PARAMETERS: {
        key: {
            "fourth_order_tensor": C,
            "bc_values": bc_values,
            "bc": bound,
            "source": source,
        }
    },
    pp.DISCRETIZATION_MATRICES: {key: {}},
}

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [5]:
# discretize and solve the system
mpsa = pp.Mpsa(key)
mpsa.discretize(sd, data)

A, b = mpsa.assemble_matrix_rhs(sd, data)
u = sps.linalg.spsolve(A, b)

Compute now the tractions. Since we are solving a rigid-body motion, we can verify if the post-computed tractions, used as surrogate of the stress tensor $\sigma$, is null.

In [6]:
# post process the traction for each face
mat = data[pp.DISCRETIZATION_MATRICES][key]
mat_stress = mat[mpsa.stress_matrix_key]
mat_bound_stress = mat[mpsa.bound_stress_matrix_key]

# The measure is in Pascals
t = mat_stress @ u + mat_bound_stress @ bc_values

# verify the stress and displacement
# assert np.allclose(t, 0) # COMMENT TO AVOID ASSERTION ERROR

In this case the stress tensor $\sigma$ is not null, in fact the Green-Lagrange tensor $E$ is null for a rigid-body motion. We have
$$
   2 E = F^\top F - I = \nabla u + \nabla u^\top + \nabla u^\top \nabla u
$$
the stress tensor si given by $2\sigma = 2C : \epsilon(u) = C : (\nabla u + \nabla u^\top)$, where the non-linear term is missing. Under the hypothesis of small deformations the stress tensor is thus not null.

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [ ]:
# reshape the displacement for the export
u_3d = np.reshape(u, (sd.dim, -1), order="F")
if dim == 2:
    u_3d = np.vstack((u_3d, np.zeros(sd.num_cells)))

save = pp.Exporter(sd, "sol", folder_name="ex1")
save.write_vtu([("u", u_3d)])

In [8]:
# Consistency check
assert np.isclose(np.linalg.norm(u), 0.025093920575932138)